In [ ]:
!pip install gdown

In [ ]:
!gdown --id 1qGWokd9ghYj2braD0R1xzCxG4EmCWrvZ
!gdown --id 1iu4OIdcheZFTw0-SehJVC18WU-U4oWeC
!unzip asar-dataset-v2.zip

# Move test data into train, and val folders

In [ ]:
import os
import shutil

def list_splitter(list_to_split, ratio):
    first_half = int(len(list_to_split) * ratio)
    return list_to_split[:first_half], list_to_split[first_half:]

source = os.path.join(os.getcwd(), 'asar-dataset-v2/test')
target1 = os.path.join(os.getcwd(), 'asar-dataset-v2/train')
target2 = os.path.join(os.getcwd(), 'asar-dataset-v2/val')

list_dir = os.listdir(source)
for sub_dir in list_dir:
    print("moving " + str(sub_dir))
    target_val_folder = os.path.join(target2, sub_dir)
    target_train_folder = os.path.join(target1, sub_dir)
    if os.path.exists(target_val_folder) and os.path.exists(target_train_folder):
        folderPath = os.path.join(source, sub_dir)
        files = os.listdir(folderPath)
        val_list, train_list = list_splitter(files, 0.5)
        for filename in val_list:
            filePath = os.path.join(folderPath, filename)
            shutil.move(filePath, target_val_folder)
        for filename in train_list:
            filePath = os.path.join(folderPath, filename)
            shutil.move(filePath, target_train_folder)

# Clean dataset misspelled

In [ ]:
import os
import shutil
import random

# Read erros.txt file in dict with old_value=key and new_value=value
words_dict = {}
file = open('errors.txt', 'r', encoding="UTF-8-sig")
Lines = file.readlines()

for line in Lines:
    words = line.strip().split("=>")
    words_dict[words[0].strip()]= words[1].strip()

# Rename old_value with new_value with merging folder if exists

folders = ['asar-dataset-v2/train', 'asar-dataset-v2/val']

for folder in folders:
    folder_path = os.getcwd() + '/' + folder
    directories = os.listdir(folder_path)
    for directory in directories:
        if directory in words_dict.keys():
            new_word = words_dict[directory]
            old_word_path = folder_path + '/' + directory
            new_word_path = folder_path + '/' + new_word
            if os.path.exists(new_word_path):
                # move files, delete this folder
                images = os.listdir(old_word_path)
                for image in images:
                    if os.path.exists(new_word_path + '/' + image):
                        file_name, ext = image.split("/")[-1].split(".")
                        r = random.randint(1, 1000)
                        modified_image = os.path.join(new_word_path, f"{file_name}_{r}.{ext}")
                        shutil.move(old_word_path + '/' + image, modified_image)
                    else:
                        shutil.move(old_word_path + '/' + image, new_word_path)
                shutil.rmtree(old_word_path)
                print("deleting " + directory)
            else:
                # just rename it
                print("renaming " + directory)
                os.rename(old_word_path, new_word_path)